In [ ]:
import sys
sys.path.append(r'C:\Users\lwang\Documents\GitHub\DataAnalysis')

In [ ]:
import numpy as np
import os
from pathlib import Path
import pathlib
import platform
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from scipy import signal 
from scipy import odr 
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from statsmodels.regression.rolling import RollingOLS

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
biolinq_col_keep = ['Algo','Study','ms_since_boot','algo_state','algo_state_desc','rate_of_change','is_rate_of_change_valid',
                    'time_from_start','biolinq','biolinq_s1','biolinq_s3','biolinq_s4','biolinq_no_blank','sensor_id','UTC_Time',
                    'color_code','color_desc','cur1','cur3','cur4','deg1_103','mag1_103','deg3_103','mag3_103','deg4_103','mag4_103',                   
                    'deg1_1007','mag1_1007','deg3_1007','mag3_1007','deg4_1007','mag4_1007','deg1_10000','mag1_10000','deg3_10000','mag3_10000','deg4_10000','mag4_10000',                   
                    ]
sensor_map_col_keep = [
    'Subject ID','IV Arm','Self Apply','Location','Config','Tier','Sorting Tier','Chemistry','Sorting Tier mfg','Chemistry mfg','Lot','Build',
    'Wafer Lot #','Wafer Type','Wafer Batch - #','Wafer Location','Cardinal','Quadrant','Edge Level','Outermost'
]
all_col_keep = biolinq_col_keep + sensor_map_col_keep

match_biolinq_ysi = True
match_biolinq_acck = True
match_biolinq_Dexcom = False
match_biolinq_Libre = False
reference_setting = {
    'ysi': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 0,
        'value_ub': 900
    },
    'acck': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 10,
        'value_ub': 600
    },
    'Libre': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 40,
        'value_ub': 400
    },
    'Dexcom': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 40,
        'value_ub': 400
    }         
}

eval_Dexcom = True
Evaluate_Dexcom_setting = {
    'ysi': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
    },
    'acck': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
    },        
}

eval_Libre = True
Evaluate_Libre_setting = {
    'ysi': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
    },
    'acck': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
    },    
}

In [ ]:
algo_data_dict = {
    # 'Isabella03':{
    #     'data_root_path': r"S:\for Brandegee Assembler\Output\6b6d8c5_gModel_indexFix_cgmadj_mfg_info",
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','Pre-Piv-2-training','Pre-Piv-2-validation','eBlinqMayStudyCtrl','eBlinqJuneStudyCtrl']
    # },        
    # 'Isabella04':{
    #     'data_root_path': r"S:\for Brandegee Assembler\Output\tree_libre2ysi_extra_gModel_all_mfg_info",
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','Pre-Piv-2-training','Pre-Piv-2-validation','eBlinqMayStudyCtrl','eBlinqJuneStudyCtrl']
    # },   
    'Isabella03_gModel_allData':{
        'data_root_path': r"S:\for Brandegee Assembler\Output\tree_libre2ysi_extra_gModel_all_mfg_info",
        'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','Pre-Piv-2-training','Pre-Piv-2-validation','eBlinqMayStudyCtrl','eBlinqJuneStudyCtrl']
    },    
    # 'mfg_model1_Keith_mdl9':{
    #     'data_root_path': r"S:\AlgorithmOutput\mdl9",
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','Pre-Piv-2-training','Pre-Piv-2-validation','MayStudy','JuneStudy_ctrl']
    # },   
    # 'nn_one_allData_training':{
    #     'data_root_path': r"S:\for Brandegee Assembler\Output\oneForAll_plus_robust_tree_AllData_ThirdAttempt",
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','Pre-Piv-2-training','Pre-Piv-2-validation','eBlinqMayStudyCtrl','eBlinqJuneStudyCtrl']
    # },   
    # 'nn_one_mdl9_training':{
    #     'data_root_path': r"S:\for Brandegee Assembler\Output\oneForAll_plus_robust_tree_mdl9TrainingData_SecondAttempt",
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','Pre-Piv-2-training','Pre-Piv-2-validation','eBlinqMayStudyCtrl','eBlinqJuneStudyCtrl']
    # },         
}
save_agg_biolinq_ref_df = True
save_agg_biolinq_datastore = True
save_agg_Dexcom_ref_df = True
save_agg_Libre_ref_df = True
folder_to_save = r'C:\Users\lwang\Documents\Simulation\data for JMP'
file_suffix = 'CompareNnModel2'
agg_biolinq_ref_df_file_name = f'agg_biolinq_ref_df_{file_suffix}.csv'
agg_biolinq_datastore_file_name = f'agg_biolinq_datastore_{file_suffix}.csv'
agg_Dexcom_ref_df_file_name = f'agg_Dexcom_ref_df_{file_suffix}.csv'
agg_Libre_ref_df_file_name = f'agg_Libre_ref_df_{file_suffix}.csv'
agg_biolinq_ref_df_file_path = os.path.join(folder_to_save,agg_biolinq_ref_df_file_name)
agg_biolinq_datastore_file_path = os.path.join(folder_to_save,agg_biolinq_datastore_file_name)
agg_Dexcom_ref_df_file_path = os.path.join(folder_to_save,agg_Dexcom_ref_df_file_name)
agg_Libre_ref_df_file_path = os.path.join(folder_to_save,agg_Libre_ref_df_file_name)

In [ ]:
head_agg_biolinq_ref_df = False
head_agg_biolinq_datastore = False
head_agg_Dexcom_ref_df = False
head_agg_Libre_ref_df = False
match_biolinq_ref = {}
match_biolinq_ref['match_biolinq_ysi'] = match_biolinq_ysi
match_biolinq_ref['match_biolinq_acck'] = match_biolinq_acck
match_biolinq_ref['match_biolinq_Dexcom'] = match_biolinq_Dexcom
match_biolinq_ref['match_biolinq_Libre'] = match_biolinq_Libre
for algo,algo_info in algo_data_dict.items():
    print(f'Processing data by algo: {algo}')
    data_root_path = algo_info['data_root_path']
    study_folder = algo_info['study_folder']
    for study in study_folder:
        print(f'Processing data from clinical trial: {study}')
        study_full_path = os.path.join(data_root_path,study)
        loaded_data = FileRead.load_pickle('alg_out.zip', study_full_path, as_dict=True)
        all_sensor_data = loaded_data['all_sensor_data']
        sensor_map_df = loaded_data['sensor_map_df']
        all_subject_id = sensor_map_df['Subject ID'].unique()
        subject_id_processed = []
        for sensor in all_sensor_data.keys():
            print(f'Processing data from sensor: {sensor}')
            current_subject_id = sensor_map_df.loc[sensor_map_df['Sensor_Id']==sensor,'Subject ID'].to_string(index=False)
            if current_subject_id in subject_id_processed:
                subject_processed_already = True
            else:
                subject_id_processed.append(current_subject_id)
                subject_processed_already = False
            current_sensor_data = all_sensor_data[sensor].copy()
            biolinq_exist = False
            ref_exist = {}
            ref_exist['acck_exist'] = False
            ref_exist['ysi_exist'] = False
            ref_exist['Libre_exist'] = False
            ref_exist['Dexcom_exist'] = False
            ref_df_dict = {}
            biolinq_ref_df_dict = {}
            if ('biolinq' in current_sensor_data.keys()) and ('biolinq_datastore' in current_sensor_data.keys()):
                biolinq_exist = True
                biolinq_df = current_sensor_data['biolinq'].copy()
                biolinq_datastore = current_sensor_data['biolinq_datastore'].copy()
                if biolinq_df.empty:
                    biolinq_exist = False
            
            if not biolinq_exist:
                continue
            biolinq_df['Algo'] = algo
            biolinq_df['Study'] = study
            biolinq_df = biolinq_df.merge(right=sensor_map_df,how='left',left_on='sensor_id',right_on='Sensor_Id')
            biolinq_datastore['Algo'] = algo
            biolinq_datastore['Study'] = study
            biolinq_datastore = biolinq_datastore.merge(right=sensor_map_df,how='left',left_on='sensor_id',right_on='Sensor_Id')

            #Fill in desired columns with NaN
            for col in all_col_keep:
                if col not in biolinq_df.columns:
                    biolinq_df[col]=None
            #Fill in desired columns with NaN
            for col in all_col_keep:
                if col not in biolinq_datastore.columns:
                    biolinq_datastore[col]=None                    
                            
            for this_ref_name,this_ref_setting in reference_setting.items():
                if this_ref_name in current_sensor_data.keys():
                    ref_exist[f'{this_ref_name}_exist'] = True
                    ref_df_dict[f'{this_ref_name}_df'] = current_sensor_data[this_ref_name].copy()
                    if ref_df_dict[f'{this_ref_name}_df'].empty:
                        ref_exist[f'{this_ref_name}_exist'] = False
                    else:
                        ref_df_dict[f'{this_ref_name}_df'].rename(columns={this_ref_name:'ref_value'},inplace=True)
                        ref_df_dict[f'{this_ref_name}_df']['ref_type'] = this_ref_name
                        valid_idx = ref_df_dict[f'{this_ref_name}_df']['ref_value'].ge(this_ref_setting['value_lb']) \
                            & ref_df_dict[f'{this_ref_name}_df']['ref_value'].le(this_ref_setting['value_ub'])
                        ref_df_dict[f'{this_ref_name}_df'] = ref_df_dict[f'{this_ref_name}_df'].loc[valid_idx,:]
                        ref_df_dict[f'{this_ref_name}_df'].loc[:,'subject_id_ref'] = current_subject_id
                if (not ref_exist[f'{this_ref_name}_exist']) or (not match_biolinq_ref[f'match_biolinq_{this_ref_name}']):
                    continue
                time_to_match = this_ref_setting['time_to_match']
                direction_to_match = this_ref_setting['direction_to_match']
                tol_to_match = this_ref_setting['tol_to_match']
                biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'] = pd.merge_asof(left=ref_df_dict[f'{this_ref_name}_df'].dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=biolinq_df[all_col_keep].dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                            on=time_to_match, direction=direction_to_match, tolerance=pd.Timedelta(tol_to_match))

                #Save matched pairs table into CSV per reference
                if save_agg_biolinq_ref_df:
                    if head_agg_biolinq_ref_df==False:
                        biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'].to_csv(agg_biolinq_ref_df_file_path,mode='a',float_format='%g',index=False,header=True)
                        head_agg_biolinq_ref_df=True
                    else:
                        biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'].to_csv(agg_biolinq_ref_df_file_path,mode='a',float_format='%g',index=False,header=False) 

            # Save datastore table into csv per sensor
            if save_agg_biolinq_datastore:
                if head_agg_biolinq_datastore==False:
                    biolinq_datastore[all_col_keep].to_csv(agg_biolinq_datastore_file_path,mode='a',float_format='%g',index=False,header=True)
                    head_agg_biolinq_datastore=True
                else:
                    biolinq_datastore[all_col_keep].to_csv(agg_biolinq_datastore_file_path,mode='a',float_format='%g',index=False,header=False)

            Dexcom_ref_df_dict = {}
            if eval_Dexcom and ref_exist['Dexcom_exist'] and (not subject_processed_already):
                for this_ref_name,this_ref_setting in Evaluate_Dexcom_setting.items():
                    if ref_exist[f'{this_ref_name}_exist']:
                        time_to_match = this_ref_setting['time_to_match']
                        direction_to_match = this_ref_setting['direction_to_match']
                        tol_to_match = this_ref_setting['tol_to_match']
                        Dexcom_ref_df_dict[f'Dexcom_{this_ref_name}_df'] = pd.merge_asof(left=ref_df_dict[f'{this_ref_name}_df'].dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=ref_df_dict['Dexcom_df'].dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                                    on=time_to_match, direction=direction_to_match, tolerance=pd.Timedelta(tol_to_match),suffixes=(None,'_Dexcom'))
                        
                        #Save matched pairs table into CSV per reference
                        if save_agg_Dexcom_ref_df:
                            if head_agg_Dexcom_ref_df==False:
                                Dexcom_ref_df_dict[f'Dexcom_{this_ref_name}_df'].to_csv(agg_Dexcom_ref_df_file_path,mode='a',float_format='%g',index=False,header=True)
                                head_agg_Dexcom_ref_df=True
                            else:
                                Dexcom_ref_df_dict[f'Dexcom_{this_ref_name}_df'].to_csv(agg_Dexcom_ref_df_file_path,mode='a',float_format='%g',index=False,header=False)       

            Libre_ref_df_dict = {}
            if eval_Libre and ref_exist['Libre_exist'] and (not subject_processed_already):
                for this_ref_name,this_ref_setting in Evaluate_Libre_setting.items():
                    if ref_exist[f'{this_ref_name}_exist']:
                        time_to_match = this_ref_setting['time_to_match']
                        direction_to_match = this_ref_setting['direction_to_match']
                        tol_to_match = this_ref_setting['tol_to_match']
                        Libre_ref_df_dict[f'Libre_{this_ref_name}_df'] = pd.merge_asof(left=ref_df_dict[f'{this_ref_name}_df'].dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=ref_df_dict['Libre_df'].dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                                    on=time_to_match, direction=direction_to_match, tolerance=pd.Timedelta(tol_to_match),suffixes=(None,'_Libre'))
                        
                        #Save matched pairs table into CSV per reference
                        if save_agg_Libre_ref_df:
                            if head_agg_Libre_ref_df==False:
                                Libre_ref_df_dict[f'Libre_{this_ref_name}_df'].to_csv(agg_Libre_ref_df_file_path,mode='a',float_format='%g',index=False,header=True)
                                head_agg_Libre_ref_df=True
                            else:
                                Libre_ref_df_dict[f'Libre_{this_ref_name}_df'].to_csv(agg_Libre_ref_df_file_path,mode='a',float_format='%g',index=False,header=False)                                                     
